In [ ]:
using CSV
using DataFrames

In [ ]:
using JuMP
using Cbc

## Exploring the Data
First, let's load our 2018-2019 player data. This CSV file contains statistics for US-born players in the 2018-2019 season.

Source: https://www.basketball-reference.com/

In [ ]:
df = CSV.read("NBA_data_2018_2019.csv");
first(df,10)

## A Simple Model

We will build our optimization models using the JuMP package in Julia: http://www.juliaopt.org/JuMP.jl/v0.19.0/

First, we have to initialize a model in JuMP using the Model() function. We also tell Julia that we are using the **Cbc** solver. This is an open-sources solver that allows us to solve integer optimization problems. Other popular alternatives are **Gurobi** and **CPLEX**, but these require licenses. 

In [ ]:
model = Model(Cbc.Optimizer)

### I. Defining the Decision Variables
First we need to create our decision variables. Remember, we want to construct variables:

$$
    x_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is selected for the team,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

These are called **binary** variables since they only take on values between 0 and 1. We need an $x_i$ for all $i=1,\ldots,N$.

In [ ]:
N = nrow(df)
@variable(model, x[1:N], Bin)

### II. Formalizing the Objective

We'll start with a basic objective, which is simply to maximize the average Player Efficiency Rating (PER) of the players on the selected team. We'll denote player $i$'s PER as $c_i$. We can then calculate the objective as:

$$ \frac{1}{N} \sum_{i=1}^{N}(x_i*s_i)$$

We can easily formulate this in JuMP:

In [ ]:
s = df[!,:PER]
@objective(model, Max, 1/12*sum(x[i]*s[i] for i=1:N));

### III. Constructing the Constraints 

Now that we have defined our variables and quantified our goal, we need to specify what requirements any team must satisfy. Let's start by just placing a constraint on team size. 

In [ ]:
@constraint(model, sum(x[i] for i=1:N) == 12);

### IV. Solving the Model
We have specified the three key elements of any mixed-integer optimization model: 
- Decision Variables
- Objective 
- Feasibility Constraints

We're ready to solve the model!

In [ ]:
optimize!(model);

Let's see what our results look like: we can look at the values of our decision variables $x_1,\ldots,x_N$ to see which indices were set to 1. These indices correspond to the players that were selected for the team.

In [ ]:
selection_indices = value.(x);

In [ ]:
selected_players = df[selection_indices.==1,:Player]

In [ ]:
objective_value(model)

How well does this match the list of finalists for the Olympics? 

In [ ]:
df[selection_indices.==1,[:Player,:Pos,:AllStar,:Olympics2016,:OlympicsFinalist]]

## Making a More Realistic Team

While this team is technically legal under Olympic regulations, it's not enough for us just to look at PER! We want to make sure that we have enough players of each position to fill out the team. We can add a few constraints to make a more useful team. 

In [ ]:
## Our original model
model2 = Model(Cbc.Optimizer)
@variable(model2, x[1:N], Bin)
@objective(model2, Max, 1/12*sum(x[i]*df[i,:PER] for i=1:N));
@constraint(model2, sum(x[i] for i=1:N) == 12);

#### Position Requirements

Suppose we want at least 4 guards, 4 forwards, and 3 centers. We first define indicators for each player's position. For example, to indicate which players are forwards, we define:


$$
    f_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is a forward,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

We can create similar variables $c_i$ and $g_i$ for center and guard positions. Note that this is **data**; these are not decision variables.

In [ ]:
c = ifelse.(df[!,:Pos].=="C",1,0);
f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);

println("Center Count: ", sum(c))
println("Forward Count: ", sum(f))
println("Guard Count: ", sum(g))

We can create constraints by summing the position indicator over the players who are selected for the team, like we did with the objective:

$$ \sum_{i=1}^{N}(f_i*x_i) \geq 4$$

We can do the same for the center and guard positions as well.

In [ ]:
# Forward constraint
@constraint(model2, sum(f[i]*x[i] for i=1:N) >= 4);

# Center constraint
@constraint(model2, sum(c[i]*x[i] for i=1:N) >= 3);

# Guard constraint
@constraint(model2, sum(g[i]*x[i] for i=1:N) >= 4);

#### Defensive Ability
We want to make sure that our team has good average defensive ability. One (imperfect!) way to measure this is using Defensive Box Plus/Minus (DBPM). We can require that the average DBPM of the chosen team is at least +1. 


$$ \frac{1}{N} \sum_{i=1}^{N}(d_i*x_i) \geq 1$$

where $d_i$ is player $i$'s DBPM. 

In [ ]:
d = df[!,:DBPM]
@constraint(model2, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1.0);

#### Experience Constraints
We also want to make sure our team has enough Olympics experience: let's add a constraint that at least 3 selected players were on the 2016 Olympic team.

We use a similar syntax as before:

$$ \sum_{i=1}^{N}(o_i*x_i) \geq 3 $$

where $o_i = 1$ if player $i$ was on the 2016 Olympic team, and 0 otherwise. 


In [ ]:
o = df[!,:AllStar]
@constraint(model2, sum(o[i]*x[i] for i=1:N) >= 3);

Let's solve our new model.

In [ ]:
set_silent(model2)
optimize!(model2)

In [ ]:
selected_players2 = sort(df[value.(x).==1,:Player])

If we compare to the original player list, we see that we have removed Montrezl Harrell (Center) and added Jimmy Butler (Forward).

In [ ]:
sort(selected_players)

How good is our new model, according to our **objective function**?

In [ ]:
objective_value(model2)

This is < 0.1 worse than our original solution: even with these new constraints, we haven't lost much.

## Variations of Our Basic Model


Before we experiment further, we can put our full model into a **function** that will let us easily modify the model for different objectives and datasets.

In [ ]:
function create_roster(df, objective_metric)
    m = Model(Cbc.Optimizer)
    set_silent(m)
    
    # Define variables
    N = nrow(df)
    @variable(m, x[1:N], Bin)
    
    # Define relevant data columns
    c = ifelse.(df[!,:Pos].=="C",1,0);
    f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
    g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);
    d = df[!,:DBPM];
    o = df[!,:AllStar];
    
    # Team size constraint
    @constraint(m, sum(x[i] for i=1:N) == 12);
    
    # Position constraints
    @constraint(m, sum(f[i]*x[i] for i=1:N) >= 4);
    @constraint(m, sum(c[i]*x[i] for i=1:N) >= 3);
    @constraint(m, sum(g[i]*x[i] for i=1:N) >= 4);

    # Experience constraint
    @constraint(m, sum(o[i]*x[i] for i=1:N) >= 3);

    # Defensive ability constraint
    @constraint(m, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1);
    
    # Objective
    @objective(m, Max, 1/12 * sum(x[i]*df[i,objective_metric] for i=1:N))
    
    optimize!(m)
    
    players = sort(df[value.(x).>0.99,:Player])
    return(objective_value(m), players)
end

Now, it is easy for us to build models for different objective functions. We simply have to specify our dataset of interest and the column of the metric that we want to maximize.

#### Alternative Objective Functions

We chose to optimize average PER, but we could have chosen other metrics too. How does our team composition change when we use other measures of player performance? 

In [ ]:
obj_per, players_per = create_roster(df,:PER);
obj_bpm, players_bpm = create_roster(df,:BPM);
obj_ws48, players_ws48 = create_roster(df,:WS_48);

#### How different is the playoff perspective?  

Now let's repeat the model (with the original PER objective), but using data from the past two playoff seasons instead. We're looking at average playoff statistics for all active US-born NBA players.

In [ ]:
df_playoffs = CSV.read("NBA_data_playoffs_2017_2019.csv");
first(df_playoffs,10)

We can leverage the same function, but now we want to speficy a new dataset, **df_playoffs**, rather than the original dataset **df**.

In [ ]:
obj_playoff, players_playoff = create_roster(df_playoffs,:PER);
players_playoff

We can create a table to compare how much these different lists overlap. We can also add a column commparing the  

In [ ]:
player_compare = join(DataFrame(Player = players_per, PER = 1), 
    DataFrame(Player = players_bpm, BPM = 1),
    DataFrame(Player = players_ws48, WS_48 = 1), 
    DataFrame(Player = players_playoff, PER_PLAYOFF = 1),
    kind = :outer, on = :Player)

sort(player_compare)

In [ ]:
# Pull a dataset of the olympic finalists
olympics_finalists = unique(df[df[!,:OlympicsFinalist].==1,[:Player,:OlympicsFinalist]])

sort(join(player_compare, olympics_finalists, on = :Player, kind = :left))

Anthony Davis, James Harden, Kawhi Leonard, and Lebron James are chosen in every variation of our model. Certain players only appear when we look at playoff statistics (e.g. Chris Paul, Draymond Green), while others look better when we focus on the regular season (e.g. Jimmy Butler).